In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1,org.apache.spark:spark-avro_2.12:3.3.1 pyspark-shell'

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.types as T
import pyspark.sql.functions as F
from pyspark.sql.functions import sum, col, desc



spark = SparkSession \
    .builder \
    .appName("Spark-Notebook") \
    .getOrCreate()
spark.sparkContext.setLogLevel('WARN')

:: loading settings :: url = jar:file:/home/jainendra/spark/spark-3.3.2-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jainendra/.ivy2/cache
The jars for the packages stored in: /home/jainendra/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5f8be0c4-f3b4-4aef-ba48-e0856aeb1834;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.1 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	fou

23/03/14 11:30:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
df_kafka_raw = spark \
      .readStream \
      .format("kafka") \
      .option("kafka.bootstrap.servers", "*************") \
      .option("subscribe", "rides_green,rides_fhv") \
      .option("startingOffsets", "earliest") \
      .option("kafka.security.protocol","SASL_SSL") \
      .option("kafka.sasl.mechanism", "PLAIN") \
      .option("kafka.sasl.jaas.config",
        """org.apache.kafka.common.security.plain.PlainLoginModule required username="************" password="************";""").load()


In [4]:
df_kafka_raw.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [5]:
df_kafka_raw

DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]

In [6]:
df_kafka_encoded = df_kafka_raw.selectExpr("CAST(key AS STRING)","CAST(value AS STRING)")

In [7]:
df_kafka_encoded.printSchema()

root
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)



In [8]:
def parse_ride_from_kafka_message(df_raw, schema):
    """ take a Spark Streaming df and parse value col based on <schema>, return streaming df cols in schema """
    assert df_raw.isStreaming is True, "DataFrame doesn't receive streaming data"

    df = df_raw.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

    # split attributes to nested array in one Column

    col = F.split(df['value'], ', ')
    #print(col)

    # expand col to multiple top-level columns
    for idx, field in enumerate(schema):
        #print (field.name,col.getItem(idx).cast(field.dataType) )
        df = df.withColumn(field.name, col.getItem(idx).cast(field.dataType))
    return df.select([field.name for field in schema])

In [9]:
#         self.dispatching_base_num = arr[1]
#         self.pickup_datetime = datetime.strptime(arr[2], "%Y-%m-%d %H:%M:%S"),
#         self.dropOff_datetime = datetime.strptime(arr[3], "%Y-%m-%d %H:%M:%S"),
#         self.PULocationID = int(eval(arr[4]))
#         self.DOLocationID = int(eval(arr[5]))
#         self.SR_Flag = arr[6]
#         self.Affiliated_base_number = arr[7]

ride_schema = T.StructType(
    [#T.StructField("dispatching_base_num", T.StringType()),
     T.StructField('pickup_datetime', T.StringType()),
     T.StructField('dropOff_datetime', T.StringType()),
     T.StructField("PULocationID", T.StringType()),
     T.StructField("DOLocationID", T.StringType()),
     #T.StructField("SR_Flag", T.StringType()),
     #T.StructField("Affiliated_base_number", T.StringType()),
     T.StructField("texi_type", T.StringType())
     ])

In [10]:
df_rides = parse_ride_from_kafka_message(df_raw=df_kafka_raw, schema=ride_schema)

In [ ]:
df_rides.printSchema()

In [ ]:
def sink_console(df, output_mode: str = 'complete', processing_time: str = '5 seconds'):
    write_query = df.writeStream \
        .outputMode(output_mode) \
        .trigger(processingTime=processing_time) \
        .format("console") \
        .option("truncate", False) \
        .start()
    return write_query # pyspark.sql.streaming.StreamingQuery

In [ ]:
write_query = sink_console(df_rides, output_mode='append')

In [ ]:
def sink_memory(df, query_name, query_template):
    write_query = df \
        .writeStream \
        .queryName(query_name) \
        .format('memory') \
        .start()
    query_str = query_template.format(table_name=query_name)
    query_results = spark.sql(query_str)
    return write_query, query_results

In [ ]:
query_name = 'frequent_pipup_location'
query_template = 'select count(distinct(PULocationID)) from {table_name}'
write_query, df_vendor_location_id = sink_memory(df=df_rides, query_name=query_name, query_template=query_template)

In [ ]:
print(type(write_query)) # pyspark.sql.streaming.StreamingQuery
write_query.status

In [ ]:
df_vendor_location_id.show()

In [ ]:
#'pickup_datetime','dropOff_datetime','PULocationID','DOLocationID','texi_type'
    
df_trip_messages = prepare_dataframe_to_kafka_sink(df=df_rides,
                                                  value_columns=["texi_type"], key_column='PULocationID')
kafka_sink_query = sink_kafka(df=df_trip_messages, topic="rides_all")

In [11]:
def sink_console(df, output_mode: str = 'complete', processing_time: str = '5 seconds'):
    write_query = df.writeStream \
        .outputMode(output_mode) \
        .trigger(processingTime=processing_time) \
        .format("console") \
        .option("truncate", False) \
        .start()
    return write_query  # pyspark.sql.streaming.StreamingQuery

def op_groupby(df, column_names):
    df_aggregation = df.groupBy(column_names).count().sort(desc("count"))
    return df_aggregation


def op_windowed_groupby(df, window_duration, slide_duration):
    df_windowed_aggregation = df.groupBy(
        F.window(timeColumn=df.pickup_datetime, windowDuration=window_duration, slideDuration=slide_duration),
        df.PULocationID
    ).count()
    return df_windowed_aggregation


def prepare_df_to_kafka_sink(df, value_columns, key_column=None):
    columns = df.columns

    df = df.withColumn("value", F.concat_ws(', ', *value_columns))
    if key_column:
        df = df.withColumnRenamed(key_column, "key")
        df = df.withColumn("key", df.key.cast('string'))
    return df.select(['key', 'value'])


def sink_kafka(df, topic):
    write_query = df.writeStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "************") \
        .option("topic", topic) \
        .option("checkpointLocation", "checkpoint") \
        .option("kafka.security.protocol","SASL_SSL") \
        .option("kafka.sasl.mechanism", "PLAIN") \
        .option("kafka.sasl.jaas.config","""org.apache.kafka.common.security.plain.PlainLoginModule required username="************" password="************";""") \
        .start()
    return write_query

In [12]:
sink_console(df_rides, output_mode='append')

df_trip_count_by_pickup_loc = op_groupby(df_rides, ['PULocationID'])
#df_trip_count_by_pickup_date_loc_id = op_windowed_groupby(df_rides, window_duration="10 minutes",slide_duration='5 minutes')

# write the output out to the console for debugging / testing
sink_console(df_trip_count_by_pickup_loc)
# write the output to the kafka topic


23/03/14 11:30:55 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-5368c314-c5e0-4c19-af65-0c8dfc467bf1. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/03/14 11:30:55 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
23/03/14 11:30:55 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-14867b39-4625-4d40-abeb-8179324dcbe9. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/03/14 11:30:55 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not support

-------------------------------------------
Batch: 0
-------------------------------------------


+-------------------------------------------+-------------------------------------------+-------------------+-------------------+-------------------+
|pickup_datetime                            |dropOff_datetime                           |PULocationID       |DOLocationID       |texi_type          |
+-------------------------------------------+-------------------------------------------+-------------------+-------------------+-------------------+
|{"pickup_datetime": ["2019-01-04 18:01:15"]|"dropOff_datetime": ["2019-01-04 18:08:43"]|"PULocationID": 175|"DOLocationID": 16 |"texi_type": "FHV"}|
|{"pickup_datetime": ["2019-01-09 09:09:08"]|"dropOff_datetime": ["2019-01-09 09:17:34"]|"PULocationID": 213|"DOLocationID": 60 |"texi_type": "FHV"}|
|{"pickup_datetime": ["2019-01-02 22:24:22"]|"dropOff_datetime": ["2019-01-02 22:44:29"]|"PULocationID": 114|"DOLocationID": 142|"texi_type": "FHV"}|
|{"pickup_datetime": ["2019-01-22 10:30:38"]|"dropOff_datetime": ["2019-01-22 10:36:00"]|"PULocation

23/03/14 11:31:07 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 12101 milliseconds


-------------------------------------------
Batch: 1
-------------------------------------------
+-------------------------------------------+-------------------------------------------+-------------------+-------------------+---------------------+
|pickup_datetime                            |dropOff_datetime                           |PULocationID       |DOLocationID       |texi_type            |
+-------------------------------------------+-------------------------------------------+-------------------+-------------------+---------------------+
|{"pickup_datetime": ["2019-01-02 18:54:14"]|"dropOff_datetime": ["2019-01-02 19:02:25"]|"PULocationID": 43 |"DOLocationID": 262|"texi_type": "GREEN"}|
|{"pickup_datetime": ["2019-01-02 18:55:10"]|"dropOff_datetime": ["2019-01-02 19:03:17"]|"PULocationID": 66 |"DOLocationID": 25 |"texi_type": "GREEN"}|
|{"pickup_datetime": ["2019-01-02 18:25:56"]|"dropOff_datetime": ["2019-01-02 18:33:17"]|"PULocationID": 244|"DOLocationID": 243|"texi_type": "

In [13]:
df_trip_count_messages = prepare_df_to_kafka_sink(df=df_rides,
                                                  value_columns=['texi_type','pickup_datetime','dropOff_datetime','PULocationID','DOLocationID'], key_column='PULocationID')
kafka_sink_query = sink_kafka(df=df_trip_count_messages, topic="rides_all")


23/03/14 11:31:20 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 2
-------------------------------------------
+-------------------------------------------+-------------------------------------------+-------------------+-------------------+---------------------+
|pickup_datetime                            |dropOff_datetime                           |PULocationID       |DOLocationID       |texi_type            |
+-------------------------------------------+-------------------------------------------+-------------------+-------------------+---------------------+
|{"pickup_datetime": ["2019-01-02 20:03:01"]|"dropOff_datetime": ["2019-01-02 20:14:00"]|"PULocationID": 74 |"DOLocationID": 263|"texi_type": "GREEN"}|
|{"pickup_datetime": ["2019-01-02 19:12:45"]|"dropOff_datetime": ["2019-01-02 19:17:33"]|"PULocationID": 129|"DOLocationID": 173|"texi_type": "GREEN"}|
|{"pickup_datetime": ["2019-01-02 19:08:41"]|"dropOff_datetime": ["2019-01-02 19:46:00"]|"PULocationID": 217|"DOLocationID": 26 |"texi_type": "

-------------------------------------------
Batch: 0
-------------------------------------------
+-------------------+-----+
|PULocationID       |count|
+-------------------+-----+
|"PULocationID": 74 |3460 |
|"PULocationID": 41 |2675 |
|"PULocationID": 7  |2397 |
|"PULocationID": 75 |2381 |
|"PULocationID": 82 |1965 |
|"PULocationID": 42 |1961 |
|"PULocationID": 129|1833 |
|"PULocationID": 264|1728 |
|"PULocationID": 181|1487 |
|"PULocationID": 255|1481 |
|"PULocationID": 97 |1421 |
|"PULocationID": 95 |1287 |
|"PULocationID": 166|1191 |
|"PULocationID": 244|1180 |
|"PULocationID": 260|1152 |
|"PULocationID": 33 |1091 |
|"PULocationID": 25 |1022 |
|"PULocationID": 80 |1000 |
|"PULocationID": 61 |979  |
|"PULocationID": 65 |936  |
+-------------------+-----+
only showing top 20 rows

23/03/14 11:31:29 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 33676 milliseconds
-------------------------------------------
Batch: 3


-------------------------------------------
Batch: 5
-------------------------------------------
+-------------------------------------------+-------------------------------------------+-------------------+-------------------+-------------------+
|pickup_datetime                            |dropOff_datetime                           |PULocationID       |DOLocationID       |texi_type          |
+-------------------------------------------+-------------------------------------------+-------------------+-------------------+-------------------+
|{"pickup_datetime": ["2019-01-04 07:11:51"]|"dropOff_datetime": ["2019-01-04 07:20:36"]|"PULocationID": 165|"DOLocationID": 21 |"texi_type": "FHV"}|
|{"pickup_datetime": ["2019-01-25 04:24:08"]|"dropOff_datetime": ["2019-01-25 04:43:12"]|"PULocationID": 119|"DOLocationID": 48 |"texi_type": "FHV"}|
|{"pickup_datetime": ["2019-01-31 21:06:14"]|"dropOff_datetime": ["2019-01-31 21:33:57"]|"PULocationID": 132|"DOLocationID": 36 |"texi_type": "FHV"}|
|{"

-------------------------------------------
Batch: 1
-------------------------------------------
+-------------------+-----+
|PULocationID       |count|
+-------------------+-----+
|"PULocationID": 74 |3503 |
|"PULocationID": 41 |2731 |
|"PULocationID": 75 |2431 |
|"PULocationID": 7  |2424 |
|"PULocationID": 82 |1998 |
|"PULocationID": 42 |1983 |
|"PULocationID": 129|1849 |
|"PULocationID": 264|1754 |
|"PULocationID": 181|1504 |
|"PULocationID": 255|1502 |
|"PULocationID": 97 |1452 |
|"PULocationID": 95 |1316 |
|"PULocationID": 166|1215 |
|"PULocationID": 244|1194 |
|"PULocationID": 260|1170 |
|"PULocationID": 33 |1118 |
|"PULocationID": 25 |1043 |
|"PULocationID": 80 |1007 |
|"PULocationID": 61 |997  |
|"PULocationID": 65 |961  |
+-------------------+-----+
only showing top 20 rows

23/03/14 11:31:45 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 15532 milliseconds
-------------------------------------------
Batch: 6


...

-------------------------------------------
Batch: 61
-------------------------------------------
+-------------------------------------------+-------------------------------------------+-------------------+-------------------+---------------------+
|pickup_datetime                            |dropOff_datetime                           |PULocationID       |DOLocationID       |texi_type            |
+-------------------------------------------+-------------------------------------------+-------------------+-------------------+---------------------+
|{"pickup_datetime": ["2019-01-03 08:51:41"]|"dropOff_datetime": ["2019-01-03 09:00:52"]|"PULocationID": 134|"DOLocationID": 197|"texi_type": "GREEN"}|
|{"pickup_datetime": ["2019-01-03 07:10:01"]|"dropOff_datetime": ["2019-01-03 07:15:59"]|"PULocationID": 244|"DOLocationID": 244|"texi_type": "GREEN"}|
|{"pickup_datetime": ["2019-01-03 08:52:26"]|"dropOff_datetime": ["2019-01-03 08:54:11"]|"PULocationID": 63 |"DOLocationID": 76 |"texi_type": 

-------------------------------------------
Batch: 62
-------------------------------------------
+-------------------------------------------+-------------------------------------------+-------------------+-------------------+---------------------+
|pickup_datetime                            |dropOff_datetime                           |PULocationID       |DOLocationID       |texi_type            |
+-------------------------------------------+-------------------------------------------+-------------------+-------------------+---------------------+
|{"pickup_datetime": ["2019-01-03 08:26:23"]|"dropOff_datetime": ["2019-01-03 08:36:51"]|"PULocationID": 166|"DOLocationID": 74 |"texi_type": "GREEN"}|
|{"pickup_datetime": ["2019-01-03 08:19:55"]|"dropOff_datetime": ["2019-01-03 08:24:50"]|"PULocationID": 134|"DOLocationID": 95 |"texi_type": "GREEN"}|
|{"pickup_datetime": ["2019-01-03 08:23:16"]|"dropOff_datetime": ["2019-01-03 08:28:11"]|"PULocationID": 42 |"DOLocationID": 42 |"texi_type": 

-------------------------------------------
Batch: 64
-------------------------------------------
+-------------------------------------------+-------------------------------------------+-------------------+-------------------+---------------------+
|pickup_datetime                            |dropOff_datetime                           |PULocationID       |DOLocationID       |texi_type            |
+-------------------------------------------+-------------------------------------------+-------------------+-------------------+---------------------+
|{"pickup_datetime": ["2019-01-03 08:32:59"]|"dropOff_datetime": ["2019-01-03 08:47:26"]|"PULocationID": 122|"DOLocationID": 121|"texi_type": "GREEN"}|
|{"pickup_datetime": ["2019-01-03 07:36:12"]|"dropOff_datetime": ["2019-01-03 07:48:18"]|"PULocationID": 74 |"DOLocationID": 238|"texi_type": "GREEN"}|
|{"pickup_datetime": ["2019-01-03 07:58:48"]|"dropOff_datetime": ["2019-01-03 08:05:33"]|"PULocationID": 41 |"DOLocationID": 42 |"texi_type": 

-------------------------------------------
Batch: 65
-------------------------------------------
+-------------------------------------------+-------------------------------------------+-------------------+-------------------+---------------------+
|pickup_datetime                            |dropOff_datetime                           |PULocationID       |DOLocationID       |texi_type            |
+-------------------------------------------+-------------------------------------------+-------------------+-------------------+---------------------+
|{"pickup_datetime": ["2019-01-03 08:25:50"]|"dropOff_datetime": ["2019-01-03 08:33:18"]|"PULocationID": 136|"DOLocationID": 127|"texi_type": "GREEN"}|
|{"pickup_datetime": ["2019-01-03 08:52:59"]|"dropOff_datetime": ["2019-01-03 08:58:31"]|"PULocationID": 116|"DOLocationID": 166|"texi_type": "GREEN"}|
|{"pickup_datetime": ["2019-01-03 08:17:18"]|"dropOff_datetime": ["2019-01-03 08:42:31"]|"PULocationID": 196|"DOLocationID": 145|"texi_type": 

-------------------------------------------
Batch: 66
-------------------------------------------
+-------------------------------------------+-------------------------------------------+-------------------+-------------------+---------------------+
|pickup_datetime                            |dropOff_datetime                           |PULocationID       |DOLocationID       |texi_type            |
+-------------------------------------------+-------------------------------------------+-------------------+-------------------+---------------------+
|{"pickup_datetime": ["2019-01-03 08:55:43"]|"dropOff_datetime": ["2019-01-03 08:57:17"]|"PULocationID": 28 |"DOLocationID": 134|"texi_type": "GREEN"}|
|{"pickup_datetime": ["2019-01-03 08:11:53"]|"dropOff_datetime": ["2019-01-03 08:58:59"]|"PULocationID": 220|"DOLocationID": 231|"texi_type": "GREEN"}|
|{"pickup_datetime": ["2019-01-03 08:20:02"]|"dropOff_datetime": ["2019-01-03 08:42:27"]|"PULocationID": 195|"DOLocationID": 97 |"texi_type": 